In [109]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1)

In [110]:
example = pd.read_csv('example.csv')

In [236]:
train_df, test_df = train_test_split(example, train_size = 0.8)
X_train = train_df.drop([ 'Grant.Status'], axis=1)
y_train = train_df['Grant.Status']
X_test = test_df.drop([ 'Grant.Status'], axis=1)
y_test = test_df['Grant.Status']

In [228]:
gbclf = GradientBoostingClassifier(n_estimators=30, max_depth=7, verbose=0,learning_rate=0.001,random_state=10)
gbclf.fit(X_train,y_train)
X_train_leaves = gbclf.apply(X_train)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbclf.fit(X_test,y_test)
X_test_leaves = gbclf.apply(X_test)[:,:,0]

In [229]:
lr = LogisticRegression(penalty='l1')
lr.fit(X_train_leaves, y_train)
y_pred_gbdtlr1 = lr.predict(X_test_leaves)
gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)

gbdt+lr auc 1: 0.70611


In [237]:
X_train_leaves=gbclf.apply(X_train)[:,:,0]
X_test_leaves=gbclf.apply(X_test)[:,:,0]

In [238]:
print(len(X_train_leaves))
print(len(X_train_leaves[0]))
y_pred_gbdtlr1 = lr.predict(X_test_leaves)
gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)

5794
30
gbdt+lr auc 1: 0.70217


In [52]:
gbc=GradientBoostingClassifier()
gbc.fit(X_train,y_train)
y_pred=gbc.predict_proba(X_test)[:,1]
auc=roc_auc_score(y_test,y_pred)
print(auc)

0.930927618648


In [53]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
y_pred_xgb=xgb.predict_proba(X_test)[:,1]
auc=roc_auc_score(y_test,y_pred_xgb)
print(auc)

0.932776749423


In [135]:
gnb= GaussianNB()
gnb.fit(X_train_leaves, y_train)
Y_pred_nb=gnb.predict_proba(X_test_leaves)[:,1]
gnb_auc = roc_auc_score(y_test,Y_pred_nb)
print('NB auc: ', gnb_auc)

NB auc:  0.657571872229


In [136]:
svc=SVC(probability=True)

svc.fit(X_train_leaves, y_train)
Y_pred_svc=svc.predict_proba(X_test_leaves)[:,1]
svc_auc=roc_auc_score(y_test,Y_pred_svc)
print('SVC auc: %.5f' % svc_auc)

SVC auc: 0.65157


In [137]:
knn=KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train_leaves, y_train)
Y_pred_knn=knn.predict_proba(X_test_leaves)[:,1]
knn_auc=roc_auc_score(y_test,Y_pred_knn)
print('KNN auc : %.5f' % knn_auc)

KNN auc : 0.80715


In [138]:
#perceptron
perceptron = Perceptron()
perceptron.fit(X_train_leaves, y_train)
Y_pred_prec=perceptron.predict(X_test_leaves)
perc_auc=roc_auc_score(y_test,Y_pred_prec)
print('Perceptron auc : ', perc_auc)

Perceptron auc :  0.633917089256


In [139]:
#linear svc
lin = LinearSVC()
lin.fit(X_train_leaves, y_train)
Y_pred_lin=lin.predict(X_test_leaves)
lin_auc=roc_auc_score(y_test,Y_pred_lin)
print('Linear SVC auc : ', lin_auc)

Linear SVC auc :  0.51237250333


In [140]:
#SGD
sgd = SGDClassifier(loss='log')
sgd.fit(X_train_leaves, y_train)
Y_pred_sgd=sgd.predict_proba(X_test_leaves)[:, 1]
sgd_auc=roc_auc_score(y_test,Y_pred_sgd)
print('SGD auc : %.5f' % sgd_auc)

SGD auc : 0.73946


In [141]:
#XGB

xgb=XGBClassifier()
xgb.fit(X_train_leaves,y_train)
Y_pred_xgb=xgb.predict(X_test_leaves)
xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
print(xgb_auc)

0.633002491391


In [142]:
lr = LogisticRegression(n_jobs=-1)
X_train_ext = hstack([X_train_leaves, X_train])
lr.fit(X_train_ext, y_train)
X_test_ext = hstack([X_test_leaves, X_test])
y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
print('gbdt+lr auc 2: %.5f' % gbdtlr_auc2)

gbdt+lr auc 2: 0.80278


In [68]:
example=pd.read_csv('train_preprocessed2.csv')

In [348]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1500)